In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import os
import time
import pickle
#from tensorflow.contrib import rnn
from io import StringIO
filename = 'timeserie_all_n_num.csv'

In [3]:
def reader(filename,ind,batchsize,lenght):
    done = False
    str_ = ''
    with open(filename) as f:
        
        str_ += next(f)
        i = 1
        try:
            while done is False:
                i +=1

                if i < ind:
                    _ = next(f)
                elif i <= ind + batchsize:
                    str_ += next(f)
                else:
                    #print("no error during reading file i:",i," ind:",ind," batchsize:",batchsize," length:",lenght)
                    done = True           
        except:
            print("Error during reading file i:",i," ind:",ind," batchsize:",batchsize," length:",lenght)
        
        return StringIO(str_)
print("done")

done


In [5]:
class fog_data:
    def __init__(self,filename,window_length = 250, batch_size = 32, split = [0.7,0.15,0.15]):
        
        self.agl = range(3,9)[:]
        print(self.agl)
        self.window_length = window_length
        self.batch_size = batch_size
        self.filename = filename
        self.filename_training = filename[:-4] + "_training.csv"
        self.filename_val = filename[:-4] + "_val.csv"        
        self.filename_test = filename[:-4] + "_test.csv"
        self.filename_dict = {}
        self.len_ = {}
        self.filename_dict["training"] = self.filename_training
        self.split = [sum(split[0:i]) for i in range(1,len(split))]
        if not os.path.isfile(self.filename_test):
            self.split_dataset()
        else:
            self.len_["training"] = 9392263
        self.filename_gan = filename[:-4] + '_gan.csv'
        if not os.path.isfile(self.filename_gan):
            self.makegan()
        else:
            self.len_["gan"] = 1159500
            self.filename_dict["gan"] = self.filename_gan
        
        
    def makegan(self):        

        df = pd.read_csv(self.filename)
        print("full data shape",df.shape,np.sum(df.FOGnFOG.values))
        
        gandata = []
        i = 50
        print(df.head(),df.iloc[0:10,8])
        while i <= (df.shape[0]-200):
            if df.FOGnFOG[i] == 1:
                #print(df.iloc[i-50:i+200,:].values.shape)
                gandata.append(df.iloc[i-50:i+200,:].values)
                i+=200
                if i% 100 == 0:
                    print("just added an extra part to the file l:",len(gandata)," i:",i)
            i += 1
            
        print(len(gandata))
        np_df = np.array(gandata).reshape((-1,10))
        print(np_df.shape,df.shape,np_df.shape[0]/df.shape[0])
        df = pd.DataFrame(data=np_df,columns=df.columns)
        self.len_["gan"] = df.shape[0]
        self.filename_dict["gan"] = self.filename_gan
        df.to_csv(self.filename_gan)
             
    def next_batch(self,type_of_data = "training", batchsize = 32*250):
        
        size = self.len_[type_of_data]
        if type_of_data == "gan":
            divider = 250
        else:
            divider = 1
        ind = int(np.floor(np.random.rand(1)*(size-batchsize)/divider))
        df = pd.read_csv(reader(self.filename_dict[type_of_data],ind*divider,batchsize,size))
        print(df.columns)
        return self.return_data(df)
    
    def val(self):
        df = pd.read_csv(self.filename_val)
        return self.return_data(df)
    def test(self):
        df = pd.read_csv(self.filename_test)
        return self.return_data(df)
    

    def split_dataset(self):
        
        filenames = [self.filename_training,self.filename_val,self.filename_test]
        with open(self.filename,'r') as f:
            self.length = sum(1 for _ in f)
        with open(self.filename,'r') as f:
            self.split.append(self.length+1)
            self.split_i = np.floor([self.length * s for s in self.split])
            print(self.split_i)
            s = 0
            i = 0
            file = open(filenames[s],'w')
            for line in f:
                if i >= self.split_i[s]:
                    file.close()
                    s += 1
                    file = open(filenames[s],'w')
                    file.write(head)
                    print("switch to ",filenames[s])
                
                if i ==0:
                    head = line
                                        
                file.write(line)
                i +=1
                #print(i)
                
            file.close()
            print("splitted the data in different files according to ", [0, self.split_i, self.length])
            self.len_ = {}
            13417517
            self.len_["training"] = self.split_i[0]
            self.len_["val"], self.len_["test"] =  self.split_i[1] - self.split_i[0], self.length - self.split_i[1]
            
    def return_data(self,df): # include self.window_size
        acc_gyr = df.iloc[:,self.agl].values
        acc_gyr = (acc_gyr-np.mean(acc_gyr))/np.std(acc_gyr)
        num_windows = int(np.floor(np.divide(acc_gyr.shape[0],(self.window_length))))
        if acc_gyr.shape[0] - num_windows*self.window_length >1:
            print("through away something!!",acc_gyr.shape[0] - num_windows*self.window_length,acc_gyr.shape[0], num_windows*self.window_length)
            print(num_windows)
        a_g  = acc_gyr[:num_windows*self.window_length,:].reshape(num_windows,self.window_length,6)
        
        fog = df.iloc[:num_windows*self.window_length,7].values
        fog_reshape = fog.reshape(num_windows,self.window_length,1)
        fognfog = np.mean(fog_reshape,axis=1,keepdims=True)
        return a_g, np.round(fognfog), fognfog
              

fog = fog_data(filename)
acc, fog_, fogi = fog.next_batch()

a, f, fi = fog.next_batch("gan")
print(acc.shape, fogi.shape,a.shape,f.shape,fi.shape)

range(3, 9)
Index(['Unnamed: 0', 'Time', 'Accelerometer_1', 'Accelerometer_2',
       'Accelerometer_3', 'Gyroscoop_1', 'Gyroscoop_2', 'Gyroscoop_3',
       'FOGnFOG', 'N_num'],
      dtype='object')
Index(['Unnamed: 0', 'Unnamed: 0.1', 'Time', 'Accelerometer_1',
       'Accelerometer_2', 'Accelerometer_3', 'Gyroscoop_1', 'Gyroscoop_2',
       'Gyroscoop_3', 'FOGnFOG', 'N_num'],
      dtype='object')
(32, 250, 6) (32, 1, 1) (32, 250, 6) (32, 1, 1) (32, 1, 1)


In [ ]:
mb_size = 32
X_dim = 250 * 6
z_dim = 5
h_dim = 512
lam = 10
n_disc = 5
lr = 1e-4

model_name = str(z_dim) + " 2 *" + str(h_dim)
print(model_name)
tf.reset_default_graph()

def xavier_init(size):
    in_dim = size[0]
    xavier_stddev = 1. / tf.sqrt(in_dim / 2.)
    return tf.random_normal(shape=size, stddev=xavier_stddev)

X = tf.placeholder(tf.float32, shape=[None, X_dim])


X = tf.placeholder(tf.float32, shape=[None, X_dim])

D_W1 = tf.Variable(xavier_init([X_dim, h_dim]))
D_b1 = tf.Variable(tf.zeros(shape=[h_dim]))

D_W2 = tf.Variable(xavier_init([h_dim, 1]))
D_b2 = tf.Variable(tf.zeros(shape=[1]))

theta_D = [D_W1, D_W2, D_b1, D_b2]


z = tf.placeholder(tf.float32, shape=[None, z_dim])

G_W1 = tf.Variable(xavier_init([z_dim, h_dim]))
G_b1 = tf.Variable(tf.zeros(shape=[h_dim]))

G_W2 = tf.Variable(xavier_init([h_dim, X_dim]))
G_b2 = tf.Variable(tf.zeros(shape=[X_dim]))

theta_G = [G_W1, G_W2, G_b1, G_b2]

def sample_z(m, n):    
    return np.random.uniform(-1., 1., size=[m, n])


def generator(z):
    G_h1 = tf.nn.relu(tf.matmul(z, G_W1) + G_b1)
    G_log_prob = tf.matmul(G_h1, G_W2) + G_b2
    #G_prob = tf.nn.sigmoid(G_log_prob)
    return G_log_prob


def discriminator(x):
    D_h1 = tf.nn.relu(tf.matmul(x, D_W1) + D_b1)
    out = tf.matmul(D_h1, D_W2) + D_b2
    return out



G_sample = generator(z)
D_real = discriminator(X)
D_fake = discriminator(G_sample)



# eps = tf.random_uniform([mb_size, 1], minval=0., maxval=1.)
# X_inter = eps*X + (1. - eps)*G_sample
# grad = tf.gradients(D(X_inter), [X_inter])[0]
# grad_norm = tf.sqrt(tf.reduce_sum((grad)**2, axis=1))
# grad_pen = lam * tf.reduce_mean((grad_norm - 1)**2)


D_loss = -tf.reduce_mean(D_real) + tf.reduce_mean(D_fake)
G_loss = -tf.reduce_mean(D_fake)

D_solver = (tf.train.RMSPropOptimizer(learning_rate=1e-4)
            .minimize(D_loss, var_list=theta_D))
G_solver = (tf.train.RMSPropOptimizer(learning_rate=1e-4)
            .minimize(G_loss, var_list=theta_G))

clip_D = [p.assign(tf.clip_by_value(p, -0.01, 0.01)) for p in theta_D]
d = tf.summary.scalar('D_loss', D_loss)
g = tf.summary.scalar('G_loss', G_loss)
                                      

merged_d = tf.summary.merge(inputs=[d])
merged_g = tf.summary.merge(inputs=[g])

train_writer_g = tf.summary.FileWriter("tensorboard_FOG" + '/train/gan_g')
train_writer_d = tf.summary.FileWriter("tensorboard_FOG" + '/train/gan_d')

sess = tf.Session()
sess.run(tf.global_variables_initializer())

if not os.path.exists('out/'):
    os.makedirs('out/')
# Create a saver.
saver = tf.train.Saver( tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES))
i = 0
start = time.time()
print("starting time:", start)

for it in range(1000001):
    for _ in range(5):
        X_mb, _, _ = fog.next_batch("gan")
        X_ = X_mb.reshape((-1,250*6))
            
        _, D_loss_curr, _, summary_ = sess.run(
            [D_solver, D_loss, clip_D,merged_d],
            feed_dict={X: X_, z: sample_z(mb_size, z_dim)}
        )
    train_writer_d.add_summary(summary_, it * 32 )
    
    _, G_loss_curr, _summary = sess.run([G_solver, G_loss, merged_g],
        feed_dict={z: sample_z(mb_size, z_dim)})
    
    train_writer_g.add_summary(_summary, it * 32 )
    
    
    if it % 100 == 0:
        print('Iter: {}; D loss: {:.4}; G_loss: {:.4}'
              .format(it, D_loss_curr, G_loss_curr))

        if it % 1000 == 0:
            print("current run time:",time.time()-start)
            samples = sess.run(G_sample, feed_dict={z: sample_z(256, z_dim)})
            saver.save(sess, 'model_save/model_' + model_name, global_step=it)
            with open(f"out/sample-{it}.pickle","wb") as f:
                pickle.dump(samples,f)
            
            #ig = plot(samples,it)
            #plt.savefig('out/{}.png'
             #           .format(str(i).zfill(3)), bbox_inches='tight')
            i += 1
            
#plt.close(fig)

5 2 *512
starting time: 1511948167.6277127
Iter: 0; D loss: -0.02077; G_loss: -0.02529
current run time: 0.6736915111541748
Iter: 100; D loss: -12.79; G_loss: 0.2572
Iter: 200; D loss: 0.5361; G_loss: -1.749
Iter: 300; D loss: -0.3359; G_loss: -1.812
Iter: 400; D loss: -0.7971; G_loss: -0.009681
through away something!! 249 7999 7750
31
Iter: 500; D loss: -4.063; G_loss: 1.405
Iter: 600; D loss: -1.519; G_loss: -0.2511
Iter: 700; D loss: -2.224; G_loss: 0.1051
through away something!! 249 7999 7750
31
Iter: 800; D loss: -0.5421; G_loss: 0.01813
Iter: 900; D loss: -1.038; G_loss: 0.2235
Iter: 1000; D loss: -0.7857; G_loss: 0.5837
current run time: 826.297043800354
through away something!! 249 7999 7750
31
Iter: 1100; D loss: -0.9108; G_loss: 0.3608
Iter: 1200; D loss: -0.8993; G_loss: 0.2702
Iter: 1300; D loss: -0.7586; G_loss: -0.01563
Iter: 1400; D loss: -0.4002; G_loss: -0.2064
Iter: 1500; D loss: -1.059; G_loss: 0.2085
through away something!! 249 7999 7750
31
Iter: 1600; D loss: -1